<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B6%5DLocal_Context%26Dynamics_Instruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Local **Context**

Local Context is helpful while managing personal seesion for each user.  We can directly pass context to tool without LLM. It available throughout the loop in all tools. You can pass a object in this context. Because **In python everything is object** so we can use list, int. str anything in context.

Step for passing context :


1. Create object using data classes .
2. Pass this object in Runner like context = your_obj
3. This will be available in tool with RunContextWrapper class
4. Context : RunContextWrapper



In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 23.5 MB/s eta 0:00:00


In [ ]:
from agents import Agent,Runner,OpenAIChatCompletionsModel,AsyncOpenAI,set_tracing_disabled, function_tool, RunContextWrapper, ModelSettings
import os
from google.colab import userdata
api_key= userdata.get('GOOGLE_API_KEY')
from dataclasses import dataclass

@dataclass
class user :
  name:str
  interest:str
  location :str

client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

import nest_asyncio
nest_asyncio.apply()

@function_tool()
async def userInfo(info :RunContextWrapper[user] ):
  print('tool user info calling')
  return f' user name is {info.context.name} and he is interested is {info.context.interest} and his location is {info.context.location}'

user1= user('Ahmed','CS','Karachi')
user2= user('Ali','IOT','New York')

math_agent = Agent(
    name='math-agent',
    instructions='''You are helpful assistant.Call userInfo and get name of person and interest.
     Start with saying Hello with name. Always resposne according to his interest and location.
     Do not always tell user their information ''',
    model=model,
    model_settings=ModelSettings(temperature=0.5),
    tools=[userInfo],
    )
# local context willl be in
reponse  = Runner.run_sync(math_agent,'Is there is any univercity in my city', context=user1)
print(reponse.final_output)

tool user info calling
Hello Ahmed!

Yes, there are many universities in Karachi. Since you're interested in Computer Science, some popular options include:

*   **NED University of Engineering & Technology:** Known for its strong engineering and CS programs.
*   **FAST-NUCES (National University of Computer and Emerging Sciences):** Highly regarded for its Computer Science and IT degrees.
*   **University of Karachi:** Offers a range of CS programs.
*   **Institute of Business Administration (IBA):** While primarily a business school, it also has a reputable Computer Science department.

Would you like me to provide more details about any of these, or perhaps help you find others that might align with your specific interests within CS?


# **Dynamic Instruction**

You can pass customize instruction for each user by these steps

1. Make a function for dynamics instruction. This function took context and agent.
2. Call this function in agent instructions

In [ ]:
from agents import Agent,Runner,OpenAIChatCompletionsModel,AsyncOpenAI,set_tracing_disabled, function_tool, RunContextWrapper, ModelSettings
import os
from google.colab import userdata
api_key= userdata.get('GOOGLE_API_KEY')
from dataclasses import dataclass

@dataclass
class user :
  name:str
  interest:str
  location :str

client : AsyncOpenAI = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model : OpenAIChatCompletionsModel = OpenAIChatCompletionsModel(
    model = 'gemini-2.5-flash',
    openai_client=client
)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

import nest_asyncio
nest_asyncio.apply()

def prompt(context :RunContextWrapper[user],agent : Agent[user]) ->str:
  print(f'Name : {context.context.name},,, Agent : {agent.name} ')
  return f'Your name is  {agent.name}. Help user {context.context.name}. '

user1= user('Ahmed','IT','Karachi')
user2= user('Ali','IOT','New York')

math_agent = Agent(
    name='math-agent',
    instructions=prompt,
    model=model,
    model_settings=ModelSettings(temperature=0.5),
    )
reponse  = Runner.run_sync(math_agent,'Tell me user name', context=user1)
print(reponse.final_output)

Name : Ahmed,,, Agent : math-agent 
Your name is Ahmed.
